In [0]:
#!pip install tensorflow==2.0.0-alpha0

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
tf.__version__

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'2.0.0-alpha0'

In [0]:
dataset = pd.read_csv("drive/My Drive/news_dataset.csv")

In [0]:
shares=dataset["Shares"]  #Target
for i in range(len(shares)):
    temp=shares[i]
    temp=temp.split(" ")
    temp=temp[0]
    if temp[-1]=="K":
        temp=float(temp[:-1])*1000
    elif temp[-1]=="M":
        temp=float(temp[:-1])*1000000
    else:
        temp=float(temp)
    shares[i]=temp
dataset["Shares"]=shares
dataset["Shares"]=dataset["Shares"].apply(lambda x: 1 if x>1500 else 0)  #Setting the threshold for the virality to 1000 shares


In [0]:
#hyperparameters
vocab_size=8000
embedding_dim=32
max_length=400
trunc_type='post'
oov_tok="<00V>"

In [18]:


articles=dataset["Article"]
articles[1620]="Nothing"

for k in range(1864):
  articles[k]=''.join([j for j in articles[k] if not j.isdigit()])
dataset["Article"]=articles


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [19]:



for i in range(len(dataset)):
    dataset["Article"][i]=re.sub("[^a-zA-Z]"," ",dataset["Article"][i])
    dataset["Article"][i]=dataset["Article"][i].lower()
    dataset["Article"][i]=dataset["Article"][i].split()
    ps=PorterStemmer()
    dataset["Article"][i]=[ps.stem(word) for word in dataset["Article"][i] if not word in set(stopwords.words("english"))]
    dataset["Article"][i]=" ".join(dataset["Article"][i])

tokenizer = Tokenizer(num_words=vocab_size,oov_token="<OOV>")
tokenizer.fit_on_texts((dataset["Article"][:]).astype(str))
word_index = tokenizer.word_index
sequences=tokenizer.texts_to_sequences((dataset["Article"][:]).astype(str))
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type,padding="post")
X=padded
y=dataset.iloc[:,3].values


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A va

In [20]:
#NN
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
model=tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
                            tf.keras.layers.GlobalAveragePooling1D(),
                           tf.keras.layers.Dropout(0.3),
                            tf.keras.layers.Dense(16,activation='relu',kernel_regularizer=tf.keras.regularizers.l2()),
                           tf.keras.layers.Dense(32,activation='relu',kernel_regularizer=tf.keras.regularizers.l2()),
                            tf.keras.layers.Dense(1,activation='sigmoid')])
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.RMSprop(),metrics=['accuracy'])
num_epochs=50
model.fit(X,
         y,
         epochs=num_epochs,
         validation_data=(X_test,y_test))


Train on 1864 samples, validate on 373 samples
Epoch 1/50
1864/1864 [==============================] - 1s 374us/sample - loss: 0.9918 - accuracy: 0.6438 - val_loss: 0.9071 - val_accuracy: 0.6193
Epoch 2/50
1864/1864 [==============================] - 0s 224us/sample - loss: 0.8330 - accuracy: 0.6491 - val_loss: 0.7967 - val_accuracy: 0.6193
Epoch 3/50
1864/1864 [==============================] - 0s 210us/sample - loss: 0.7414 - accuracy: 0.6491 - val_loss: 0.7252 - val_accuracy: 0.6193
Epoch 4/50
1864/1864 [==============================] - 0s 218us/sample - loss: 0.6876 - accuracy: 0.6491 - val_loss: 0.6868 - val_accuracy: 0.6193
Epoch 5/50
1864/1864 [==============================] - 0s 217us/sample - loss: 0.6610 - accuracy: 0.6491 - val_loss: 0.6716 - val_accuracy: 0.6193
Epoch 6/50
1864/1864 [==============================] - 0s 210us/sample - loss: 0.6514 - accuracy: 0.6491 - val_loss: 0.6677 - val_accuracy: 0.6193
Epoch 7/50
1864/1864 [==============================] - 0s 221us/

In [21]:
y_pred=model.predict(X_test)
for i in range(len(y_pred)):
  if y_pred[i]>=0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[225,   6],
       [ 19, 123]])